# Regularization
正则化的目的是为了防止模型过拟合，过拟合的模型在训练集上表现优异但是测试集上却不理想。构造模型最终目的是让模型在面对新数据的时候，可以有很好的表现。



## 向量范数
### 定义

如果向量$X\in R^n$的某个实值函数$f(X) = \|X\|$满足：

1. 正定性: $\|X\| \ge 0$, 当且仅当$X = 0$时$\|X\| = 0$
2. 齐次性: 对任意认识$\alpha$, 都有$\|\alpha X\| = |\alpha|\ \|X\|$
3. 三角不等式: 对任意$x, y \in R^n$, 都有$\|x + y\| < \|x\| + \|y\|$

则称$\|X\|$为$R^n$上的一个向量范数

### L-P范数

L-P范数不是一个范数，而是一组范数，其定义如下： 
$$
L_p = \|X\|_p = \displaystyle\sqrt[p]{\sum_{i = 1}^n|x_i|^p}, X = (x_1, x_2,\ldots, x_n)\tag{1}
$$

### L1范数

根据公式(1)的定义，L1范数可以写成如下形式
$$
L1 = \|X\|_1 = \displaystyle\sum_{i = 1}^n|x_i|\tag{2}
$$

### L2范数

2-范数也称为Euclid范数（欧几里得范数，常用计算向量长度），即向量元素绝对值的平方和再开方，表示x到零点的欧式距离
$$
L2 = \|X\|_2 = \sqrt{\displaystyle\sum_{i = 1}^nx_i^2}\tag{3}
$$

### L$\infty$范数

$$
L\infty = \|X\|_{\infty} = \underset{1\le i \le n}{\max}|x_i|\tag{4}
$$

当L-P范数中的P趋于正无穷大的时候，通过极限求解(高数内容)可以证明公式(3)

### L0范数

$$
L0 = \|X\|_0 = \displaystyle\sum_{i = 1}^nI(x_i \not= 0)\tag{5}
$$

L0范数不满足三角不等式和齐次性，故L0不是严格的范数。

## 使用范数正则化
通常，我们假设模型的损失函数为$L = f(\theta)$

常见的几种正则化方法如下

### L1-Norm

使用L1正则的时候，损失函数变成如下形式
$$
L = f(\theta) + C\|\theta\|_1 \tag{6}
$$
其中的$C$是一个超参数。L1正则可以让模型产生一个稀疏解，这个在特征选择工程的时候有很大帮助。

产生系数解的原因如下,对于公式(6)，是等价于
$$
\begin{aligned}
&\underset{\theta}{arg\ min} f(\theta)\\
&s.t.\ \|\theta\|_1 \le \eta
\end{aligned}
\tag{7}
$$
通过图像法，可以比较容易观测到解出现在约束条件的顶点。还可以参考[Likehood](../math/likehood.ipynb)

![L1-norm](imgs/re1.jpeg)

L1正则还有一个特点，在0出不可导，对梯度下降法求解问题造成极大困扰，通常使用如下的公式代替
$$
\|\theta\|_1 = \displaystyle\sum_{d}^D\sqrt{\theta^2_d + \epsilon} \tag{8}
$$

### L2-Norm

使用L2正则的时候，损失函数变成如下形式
$$
L = f(\theta) + C\|\theta\|^2_2 \tag{9}
$$
它可以将模型的参数限制在一个较小的范围，比如在SVM或者Logistic Regression中，用来分类的线性函数的参数会随着迭代次数多增加变得越来越大。可以证明，公式(9)等价于
$$
\begin{aligned}
&\underset{\theta}{arg\ min} f(\theta)\\
&s.t.\ \|\theta\|_2^2 \le \eta
\end{aligned}
\tag{10}
$$

## Dropout

dropout 是一种计算方便但功能强大的正则化方法，适用于神经网络。其基本步骤是在每一次的迭代中，随机使一部分节点失效，只训练剩下的节点。每次迭代都会随机删除，每次迭代删除的节点也都不一样，相当于每次迭代训练的都是不一样的网络，通过这样的方式降低节点之间的关联性以及模型的复杂度，从而达到正则化的效果。这点上有点类似 bagging，但是远比 bagging 来的简单。

dropout只需要设置一个超参数 keep_prob，这个参数的意义是每层节点随机保留的比例，比如将 keep_prob 设置为 0.7，那么就会随机30%的节点消失，消失这个动作其实很简单，只是将这一层的参数矩阵与根据 keep_prob 生成的 {0, 1} 矩阵做 **逐点乘积**，当然前向传播与反向传播都需要做以上的操作。

设随机变量$\xi_i$取值为0和1的概率分别为p和1-p，那么有
$$E(\xi_i) = 1-p$$<br>
dropout的处理公式为
$$h'_i = \frac{\xi_i}{1 -p}h_i$$,
则$E(h'_i) = \frac{E(\xi_i)}{1 -p}h_i = h_i$
dropout的缺点在于，需要将训练集分为不同子集输入每一次的迭代，这就需要较大的训练集，所以在训练集较小的情况下，dropout的效果可能并不好。

## Early Stopping

提前终止可能是最简单的正则化方式，他适用于模型的表达能力很强的时候。这种情况下，一般训练误差会随着训练次数的增多逐渐下降，而测试误差则会先下降而后再次上升。我们需要做的就是在测试误差最低的点停止训练即可。


## 增大训练集

更大数量的训练集是提升机器学习模型泛化能力最好的方法。在实际项目中，获取训练数据的成本会很高，这时候就需要我们自己来“创造”数据。希望在以后，GAN可以成功的应用到训练集增强领域。